# 🎮 Battle City + DreamerV3
**Runtime:** GPU (T4)

In [ ]:
#@title 1. Установка
!pip install nes-py gymnasium jax[cuda12] jaxlib ninjax elements portal ruamel.yaml chex -q

# Патч nes-py для NumPy 2.0
rom_path = "/usr/local/lib/python3.12/dist-packages/nes_py/_rom.py"
with open(rom_path, 'r') as f: content = f.read()
content = content.replace("self.prg_rom_start + self.prg_rom_size * 2**10", "int(self.prg_rom_start) + int(self.prg_rom_size) * 1024")
content = content.replace("self.chr_rom_start + self.chr_rom_size * 2**10", "int(self.chr_rom_start) + int(self.chr_rom_size) * 1024")
with open(rom_path, 'w') as f: f.write(content)

print('✅ Готово')

In [ ]:
#@title 2. Клонирование
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /content/battle-city
!git clone https://github.com/Nikolay1221/battle-city-dreamer.git /content/battle-city
print('✅ Готово')

In [ ]:
#@title 3. Тренировка 🚀
import os
os.chdir('/content/battle-city')
os.makedirs('/content/drive/MyDrive/battlecity_dreamer/logs', exist_ok=True)

!python dreamerv3/main.py \
    --task battlecity_stage0 \
    --configs battlecity size12m \
    --logdir /content/drive/MyDrive/battlecity_dreamer/logs \
    --run.steps 5000000 \
    --run.envs 8 \
    --jax.platform cuda

In [ ]:
#@title 4. TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/battlecity_dreamer/logs